In [1]:
import pymongo
import json
from sshtunnel import SSHTunnelForwarder

In this notebook there are the code chunks used for downloading the data stored in the VM collection and the ones useful for inserting the data in another instance. The recommended way for exporting and importing data on MongoDB is through the "mongoimport" and "mongoexport" utilities. We have chosen to use a programmatic approach because it helps us to filter the data we'd like to export: in particular we just wanted to export data stored before the 28th of June because that was the week when we wrote the project report.

# Download data from VM collections

### SSH Tunnel

In [2]:
with open('../keys.json') as k:
    keys = json.loads(k.read())
    MONGO_HOST = keys["SSH"]["MONGO_HOST"] # i.e. 10.9.13.14
    MONGO_DB = keys["SSH"]["MONGO_DB"] # i.e. dm_project 
    MONGO_USER = keys["SSH"]["MONGO_USER"] # i.e. studente
    MONGO_PASS = keys["SSH"]["MONGO_PASS"] # i.e. la password della vm

server = SSHTunnelForwarder(
    MONGO_HOST,
    ssh_username=MONGO_USER,
    ssh_password=MONGO_PASS,
    remote_bind_address=('127.0.0.1', 27017)
)

In [40]:
server.start() # remember to stop

In [48]:
client = pymongo.MongoClient('127.0.0.1', server.local_bind_port) # server.local_bind_port is assigned local port
db = client[MONGO_DB]
db.list_collection_names()

['system.indexes', 'twitch', 'twitter', 'vgchartz']

The import of the dumps function is useful for converting ObjectID and datetime structures in JSON-Compatible ones

In [5]:
from bson.json_util import dumps

### Export twitch data

The Twitch collection is by far the biggest one on our project. For this reason we will work on smaller subsets, so that it will be possible to download the collection files without incurring in Memory Errors.

In [7]:
pipeline = [{ "$match" : {'timestamp' : {'$lt' : '2019-06-15'}}}]
twitch_cursor = db.twitch.aggregate(pipeline)


In [8]:
twitch_dump_before_06_15 = dumps(twitch_cursor)

In [14]:
with open("twitch_before_06_15.json", "w") as f:
        f.write(twitch_dump_before_06_15)

In [15]:
pipeline_15_20 = [{ "$match" : {'$and' : [
                {'timestamp' : {'$gte' : '2019-06-15'}},
                {'timestamp' : {'$lt' : '2019-06-20'}}
            ]}}]
twitch_cursor_15_20 = db.twitch.aggregate(pipeline_15_20)
twitch_dump_15_20 = dumps(twitch_cursor_15_20)
with open("twitch_06_15_20.json", "w") as f:
        f.write(twitch_dump_15_20)

In [17]:
pipeline_20_24 = [{ "$match" : {'$and' : [
                {'timestamp' : {'$gte' : '2019-06-20'}},
                {'timestamp' : {'$lt' : '2019-06-24'}}
            ]}}]
twitch_cursor_20_24 = db.twitch.aggregate(pipeline_20_24)
twitch_dump_20_24 = dumps(twitch_cursor_20_24)
with open("twitch_06_20_24.json", "w") as f:
        f.write(twitch_dump_20_24)

In [19]:
del twitch_dump_20_24

In [20]:
pipeline_24_28 = [{ "$match" : {'$and' : [
                {'timestamp' : {'$gte' : '2019-06-24'}},
                {'timestamp' : {'$lt' : '2019-06-28'}}
            ]}}]
twitch_cursor_24_28 = db.twitch.aggregate(pipeline_24_28)
twitch_dump_24_28 = dumps(twitch_cursor_24_28)
with open("twitch_06_24_28.json", "w") as f:
        f.write(twitch_dump_24_28)

In [32]:
del twitch_dump_24_28

### Export twitter collection

In [49]:
db.twitter.find_one()

{'_id': ObjectId('5d0fa455b2b670923ac1437d'),
 'retweets': 0,
 'text': "@Didact343 I spent far too long considering this. Went with Cyberpunk as it's a totally new IP and Death Stranding… https://t.co/jN5UmcffS0",
 'user_followers': 450,
 'language': 'en',
 'query': 'death stranding',
 'username': 'Biggus Bennus',
 'likes': 1,
 'date': datetime.datetime(2019, 6, 13, 23, 58, 57),
 'user_location': 'Cardiff, Wales'}

In [44]:
from datetime import datetime
end = datetime(2019, 6, 28, 0, 0, 0)

In [58]:
pipeline = [{ "$match" : {'date' : {'$lt' : end}}}]
twitter = db.twitter.aggregate(pipeline)
twitter_dump = dumps(twitter)
with open("twitter.json", "w") as f:
        f.write(twitter_dump)

**Rember to close the connection!**

In [61]:
server.stop()

# Insert download data on another MongoDB Instance

## Important: change all IP and path according to your sistem configuration. Do not change the DB and collections names, otherwise the other notebooks won't work

In [ ]:
client = pymongo.MongoClient("localhost", 27017)
db = client["dm_project"]
db.list_collection_names()

In [24]:
from bson.json_util import loads

In [25]:
def load_json_dump(fn):
    with open(fn) as f:
        collection = loads(f.read())
    return collection


### Twitch

In [26]:
twitch_files = ["twitch_before_06_15.json", "twitch_06_15_20.json", "twitch_06_20_24.json", "twitch_06_24_28.json"]

In [27]:
for file in twitch_files:
    print(file)
    collection = load_json_dump(file)
    print("Loaded")
    db.twitch.insert_many(collection)
    print("Inserted!")

twitch_before_06_15.json
Loaded
Inserted!
twitch_06_15_20.json
Loaded
Inserted!
twitch_06_20_24.json
Loaded
Inserted!
twitch_06_24_28.json
Loaded
Inserted!


Check if data was loaded correctly

In [28]:
db.twitch.find_one()

{'_id': ObjectId('5d13a892f0bd0624b4bd34c1'),
 'timestamp': '2019-05-27 18:46:09.148466',
 'data': [{'game': {'name': 'Fortnite',
    'popularity': 171268,
    '_id': 33214,
    'giantbomb_id': 37030,
    'box': {'large': 'https://static-cdn.jtvnw.net/ttv-boxart/Fortnite-272x380.jpg'},
    'logo': {'large': 'https://static-cdn.jtvnw.net/ttv-logoart/Fortnite-240x144.jpg'},
    'localized_name': 'Fortnite',
    'norm_name': 'fortnite',
    'unary_name': 'fortnite'},
   'viewers': 181860,
   'channels': 14657},
  {'game': {'name': 'Grand Theft Auto V',
    'popularity': 121282,
    '_id': 32982,
    'giantbomb_id': 36765,
    'box': {'large': 'https://static-cdn.jtvnw.net/ttv-boxart/Grand%20Theft%20Auto%20V-272x380.jpg'},
    'logo': {'large': 'https://static-cdn.jtvnw.net/ttv-logoart/Grand%20Theft%20Auto%20V-240x144.jpg'},
    'localized_name': 'Grand Theft Auto V',
    'norm_name': 'grand theft auto v',
    'unary_name': 'grand theft auto v'},
   'viewers': 134132,
   'channels': 1571},

In [34]:
db.twitch.count()

11323

# Insert Twitter Data

In [59]:
twitter_collection = load_json_dump("twitter.json")

In [64]:
db.twitter.insert_many(twitter_collection)

In [65]:
db.twitter.find_one()

{'_id': ObjectId('5d0fa455b2b670923ac1437d'),
 'retweets': 0,
 'text': "@Didact343 I spent far too long considering this. Went with Cyberpunk as it's a totally new IP and Death Stranding… https://t.co/jN5UmcffS0",
 'user_followers': 450,
 'language': 'en',
 'query': 'death stranding',
 'username': 'Biggus Bennus',
 'likes': 1,
 'date': datetime.datetime(2019, 6, 13, 23, 58, 57),
 'user_location': 'Cardiff, Wales'}